In [59]:
import pandas as pd
from ase.visualize import view as view_molecule
from ase.io import read as read_molecule
import ase
import numpy as np
import random


from ase.cell import Cell
from dscribe.descriptors import CoulombMatrix, SineMatrix, EwaldSumMatrix, MBTR

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy.linalg as LA
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import SVR

import catboost as cb
import shap
from sklearn.inspection import permutation_importance

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error, r2_score


import warnings
warnings.filterwarnings('ignore')


In [60]:
DATA_PATH = './nomad2018-predict-transparent-conductors'


In [61]:
def custom_converter(entry):
    return np.array([float(x) for x in entry[1:-1].split(',')])

In [62]:
train_all_data = pd.read_csv(
    f'{DATA_PATH}/train_extrainfo.csv',
    converters={
        'CoulombMatrix':custom_converter,
        'SineMatrix':custom_converter,
        'EwaldSumMatrix':custom_converter
    }
)
test_all_data = pd.read_csv(
    f'{DATA_PATH}/test_extrainfo.csv',
    converters={
        'CoulombMatrix':custom_converter,
        'SineMatrix':custom_converter,
        'EwaldSumMatrix':custom_converter
    }
)

In [63]:
train_all_data['CoulombMatrix'][0].shape

(6400,)

In [64]:
#train = train.reset_index(drop=True)
vector = np.vstack((train_all_data[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values,
                    test_all_data[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']].values))

pca = PCA().fit(vector)
train_all_data['vector_pca0'] = pca.transform(train_all_data[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]
test_all_data['vector_pca0'] = pca.transform(test_all_data[['lattice_vector_1_ang', 'lattice_vector_2_ang','lattice_vector_3_ang']])[:, 0]

train_all_data.head()

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev,CoulombMatrix,SineMatrix,EwaldSumMatrix,vector_pca0
0,1,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.0680,3.4387,"[121.10109815785134, 75.93020649243516, 122.09...","[1897.7459337097173, 175.12757587921956, 73.28...","[-147.16586613403337, -83.60940041075736, 50.8...",2.303089
1,2,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.2490,2.9210,"[106.316346316413, 155.73823639588636, 139.218...","[1897.7459337097173, 155.40184168757955, 179.3...","[31.62494082794717, 499.7677140971106, 499.777...",-10.413997
2,3,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.1821,2.7438,"[76.85202206242872, 82.658215573199, 36.006484...","[1897.7459337097173, 64.86894200257981, 71.405...","[-105.23185191282306, -230.0887946397426, 9.15...",-1.223099
3,4,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,0.2172,3.3492,"[224.16370782409976, 216.10586898126473, 94.14...","[5694.30331076094, 475.2322348410168, 284.1925...","[-226.29548002634016, -767.1897774756807, -359...",-4.410966
4,5,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,0.0505,1.3793,"[161.3156894201164, 419.0476738666981, 161.311...","[5694.30331076094, 94.27325853815663, 195.3521...","[44.77486350842855, -804.0787210072267, -198.3...",-10.700240


# Basic linear regression

In [65]:
def get_eigenspectrum(matrix):
    spectrum = LA.eigvalsh(matrix)
    spectrum = np.sort(spectrum)[::-1]
    return spectrum

In [66]:
def create_matrix_df(data,pca_components=None,train=True):
    ewald_spectrum_list = []
    for m in data['SineMatrix']:
        ewald_spectrum_list.append(
            get_eigenspectrum(
                np.reshape(m, (80, 80))
            )
        )
    ewald_spectrum_df = pd.DataFrame(ewald_spectrum_list).astype(float)
    ewald_spectrum_df = ewald_spectrum_df.fillna(0)
    x = ewald_spectrum_df.loc[:, :].values
    x = StandardScaler().fit_transform(x)
    #y = data.loc[:, ['formation_energy_ev_natom']].values
    pca = PCA(n_components=15).fit(x)

    # PCA n_components calculation
    rolling_sum = 0
    n_components = 1
    for i, num in enumerate(pca.explained_variance_ratio_):
        rolling_sum += num
        if rolling_sum > 0.95:
            n_components = i
            break
    
    if not train:
        n_components = pca_components

    n_components = 80
    # print(f'Performing PCA with {n_components} components')
    pca = PCA(n_components)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data=principalComponents)
    if train:
        to_drop = ['id', 'formation_energy_ev_natom', 'bandgap_energy_ev',
               'CoulombMatrix', 'SineMatrix', 'EwaldSumMatrix']
        dfcombined = pd.concat([data, principalDf], axis=1).drop(to_drop, axis=1)
        #dfcombined = data.drop(to_drop, axis=1)
    else:
        to_drop = ['id',
               'CoulombMatrix', 'SineMatrix', 'EwaldSumMatrix']
        dfcombined = pd.concat([data, principalDf], axis=1).drop(to_drop, axis=1)
        #dfcombined = data.drop(to_drop, axis=1)
    return dfcombined, n_components



In [80]:
test_all_data.head()

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,CoulombMatrix,SineMatrix,EwaldSumMatrix,vector_pca0
0,1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,90.0006,"[96.9049560437187, 99.19361728536755, 71.84957...","[5694.30331076094, 413.6505459791896, 397.2255...","[-348.6037484968802, 64.24345052136955, 92.703...",2.433184
1,2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,90.0015,"[148.386446017212, 232.6504724629538, 112.5415...","[1897.7459337097173, 86.01846557365451, 166.92...","[-148.00045062969014, -70.36198549012532, -84....",2.291263
2,3,167,30.0,0.6667,0.1667,0.1667,4.9811,4.9808,13.4799,89.9900,90.0109,120.0014,"[450.0919801267003, 5694.30331076094, 452.0152...","[5694.30331076094, 222.76947026954662, 314.388...","[-226.78504034750154, -527.8912199191307, -490...",-4.392688
3,4,12,80.0,0.5625,0.4375,0.0000,24.3370,6.0091,5.7620,89.9995,103.8581,90.0002,"[312.0848530849396, 111.73463601708971, 157.21...","[1897.7459337097173, 76.67173856528522, 76.671...","[24.9843352404514, 490.26977981852474, -309.25...",15.025909
4,5,12,80.0,0.1875,0.5000,0.3125,24.6443,6.2906,6.1589,90.0000,104.5929,90.0001,"[81.88679558856255, 119.4441168826615, 243.051...","[5694.30331076094, 686.1471949873904, 123.5787...","[21.684939392750834, 1095.1490080698686, -193....",14.988648


In [83]:

def compute_final_values(train_all_data, test_all_data, target_column, model):
    # # train
    # dfcombined, n_components = create_matrix_df(train_all_data)
    # X_train, X_test, y_train, y_test = train_test_split(dfcombined, train_all_data[target_column], test_size = 0.30, random_state=1)
    to_drop = ['id', 'formation_energy_ev_natom', 'bandgap_energy_ev',
               'CoulombMatrix', 'SineMatrix', 'EwaldSumMatrix']
    copy_best_train = train_all_data
    
    copy_best_train = copy_best_train.drop(to_drop, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(copy_best_train, train_all_data[target_column], test_size = 0.30, random_state=1)
    rf = model.fit(X_train, y_train)
    score = rf.score(X_test, y_test)
    y_pred = rf.predict(X_test)

    print("Score:", score)
    print("Error rate:", ((mean_squared_error(y_test, y_pred)*100)), "%")
    
    # print(f'Training score for {target_column}: {score}')
    # test
    #dfcombined, n_components = create_matrix_df(test_all_data, train=False, pca_components=n_components)
    to_drop2 = ['id',
               'CoulombMatrix', 'SineMatrix', 'EwaldSumMatrix']
    copy_best_test = test_all_data
    copy_best_test = copy_best_test.drop(to_drop2, axis=1)
    predicted = rf.predict(copy_best_test)
    return predicted


## Experiments

In [91]:
models = [
    # ('randomforest', RandomForestRegressor(n_estimators=1000, random_state=2), RandomForestRegressor(n_estimators=1000, random_state=2)),
    # ('ridge', Ridge(alpha=0.001), Ridge(alpha=0.001)),
    # ('lasso', Lasso(alpha=0.001, max_iter=1e5), Lasso(alpha=0
    # ..001, max_iter=1e5)),
    # ('linear regression', LinearRegression(),LinearRegression()),
    ('catboost', cb.CatBoostRegressor(iterations=1200,
                            learning_rate=0.03,
                            depth=4,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=99,
                            od_type='Iter',
                            od_wait=50), cb.CatBoostRegressor(iterations=1200,
                            learning_rate=0.03,
                            depth=4,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=99,
                            od_type='Iter',
                            od_wait=50))
    # ('svr', SVR(kernel = 'linear'), SVR(kernel = 'linear'))
]

In [92]:
# for name, train_model, test_model in models:
#     print(f'Running {name}')

#     pred_full_test_cat_feen = 0
#     mse_cat_list_feen=[]
#     kf = model_selection.KFold(n_splits=10, shuffle=True, random_state=30)
#     for dev_index, val_index in kf.split(train_all_data):
#         dev_X, val_X = X.loc[dev_index], X.loc[val_index]
#         dev_y, val_y = Y_feen.loc[dev_index], Y_feen.loc[val_index]
#         y_pred_feen,rmsle_feen,y_pred_test_feen=runCatBoost(dev_X, dev_y, val_X, val_y,test,depth=4)
#         mse_cat_list_feen.append(rmsle_feen)
#         pred_full_test_cat_feen = pred_full_test_cat_feen + y_pred_test_feen
#     mse_cat_feen_mean=np.mean(mse_cat_list_feen)
#     print("Mean cv score : ", np.mean(mse_cat_feen_mean))
#     y_pred_test_feen=pred_full_test_cat_feen/10

#     pred_fe = compute_final_values(
#         train_all_data, 
#         test_all_data, 
#         target_column='formation_energy_ev_natom',
#         model = train_model
#     )
#     pred_bandgap = compute_final_values(
#         train_all_data, 
#         test_all_data, 
#         target_column='bandgap_energy_ev',
#         model = test_model
#     )
#     id_1 = np.arange(1, len(pred_fe)+1, 1, dtype=int)
#     submission_df = pd.DataFrame({'id':id_1,'formation_energy_ev_natom':pred_fe,'bandgap_energy_ev':pred_bandgap})
#     submission_df.to_csv(f"submissions/trial_submission_df_{name}.csv", index=False)
    

In [93]:
for name, train_model, test_model in models:
    print(f'Running {name}')
    pred_fe = compute_final_values(
        train_all_data, 
        test_all_data, 
        target_column='formation_energy_ev_natom',
        model = train_model
    )
    pred_bandgap = compute_final_values(
        train_all_data, 
        test_all_data, 
        target_column='bandgap_energy_ev',
        model = test_model
    )
    id_1 = np.arange(1, len(pred_fe)+1, 1, dtype=int)
    submission_df = pd.DataFrame({'id':id_1,'formation_energy_ev_natom':pred_fe,'bandgap_energy_ev':pred_bandgap})
    submission_df.to_csv(f"submissions/trial_submission_df_{name}.csv", index=False)
    

Running catboost
0:	learn: 0.3986521	total: 2.77ms	remaining: 3.32s
1:	learn: 0.3915828	total: 19.1ms	remaining: 11.5s
2:	learn: 0.3846775	total: 21.2ms	remaining: 8.46s
3:	learn: 0.3779850	total: 32.3ms	remaining: 9.64s
4:	learn: 0.3714261	total: 33.4ms	remaining: 7.98s
5:	learn: 0.3650647	total: 36.9ms	remaining: 7.35s
6:	learn: 0.3588528	total: 39.4ms	remaining: 6.71s
7:	learn: 0.3528597	total: 40.4ms	remaining: 6.02s
8:	learn: 0.3470102	total: 43.2ms	remaining: 5.72s
9:	learn: 0.3413066	total: 47.8ms	remaining: 5.69s
10:	learn: 0.3357712	total: 60.1ms	remaining: 6.5s
11:	learn: 0.3303934	total: 65.8ms	remaining: 6.51s
12:	learn: 0.3251103	total: 76.9ms	remaining: 7.02s
13:	learn: 0.3199451	total: 78.3ms	remaining: 6.64s
14:	learn: 0.3149054	total: 87.8ms	remaining: 6.94s
15:	learn: 0.3099854	total: 92.9ms	remaining: 6.87s
16:	learn: 0.3052104	total: 95.9ms	remaining: 6.67s
17:	learn: 0.3005546	total: 99.2ms	remaining: 6.51s
18:	learn: 0.2959874	total: 105ms	remaining: 6.52s
19:	lea

### Code to save submission

In [75]:
id_1 = np.arange(1, len(pred_fe)+1, 1, dtype=int)
submission_df = pd.DataFrame({'id':id_1,'formation_energy_ev_natom':pred_fe,'bandgap_energy_ev':pred_bandgap})
submission_df.head()


,id,formation_energy_ev_natom,bandgap_energy_ev
0,1,0.198081,1.498128
1,2,0.084218,3.706897
2,3,0.143409,3.511409
3,4,0.032686,3.010057
4,5,0.147326,1.421471


In [76]:
submission_df.to_csv("submissions/trial_submission_df_3.csv", index=False)

In [77]:
dfcombined_1, n_components_1 = create_matrix_df(train_all_data)

In [73]:
dfcombined_1.head()

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,...,70,71,72,73,74,75,76,77,78,79
0,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,...,-0.003700,-0.002907,0.004438,0.001137,0.001608,-0.012608,0.002607,-0.000743,-0.001971,-0.004777
1,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,...,0.000900,0.010686,-0.014056,-0.007346,0.003003,0.008881,0.006281,-0.009465,-0.005478,-0.005299
2,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,...,-0.000182,-0.001170,-0.001962,0.000589,-0.000964,-0.000659,0.000600,0.000013,-0.000809,0.000060
3,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,...,-0.000245,0.000454,-0.000142,0.001563,0.000906,0.000429,0.000110,0.000496,-0.000037,0.000061
4,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,...,0.003742,-0.026200,0.005514,-0.000055,-0.007035,-0.003736,-0.003962,0.004179,-0.011264,0.004670


In [74]:
X_train, X_test, y_train, y_test = train_test_split(dfcombined_1, train_all_data['formation_energy_ev_natom'], test_size = 0.30, random_state=1)
rf_c2 = RandomForestRegressor(n_estimators=1000, random_state=2).fit(X_train, y_train)
y_pred = rf_c2.predict(X_test)
print("Error rate:", ((mean_squared_error(y_test, y_pred)*100)), "%")

Error rate: 0.1337108850617039 %
